In [4]:
import igl
import scipy
import numpy as np
import torch
import open3d
import glob
from numpy.random import default_rng
from matplotlib import pyplot as plt
from numpy.random import default_rng

In [5]:
def readLbl(size,fileName):
    #takes in file name, returns the labels as an array
    file1 = open(fileName, 'r')
    Lines = file1.readlines()

    count = 0

    lbls = np.empty([size])
    # Strips the newline character
    for line in Lines:
        count += 1
        text = line.strip()[1:].split('.')
        text[1] = text[1].split(' ')[1]
        lbls[int(text[0])] = int(text[1])
        #print("Line{}: {}".format(count, )))
    return lbls

def lbl_2_lblMat(lbls):
    #takes in the labels as an array, returns an adjacency matrix between objects of same label
    labelMat = igl.all_pairs_distances(lbls,lbls,False)
    labelMat = np.where(labelMat > 0.5, 1, 0)
    return labelMat

def compareGroups(grpa,grpb,lbl,disSMat):
    #takes in two group indices, labels and dissimiliarity matrix, returns how similar they are as two groups
    grpA_idx = np.reshape(np.where(lbl == grpa),(20,))
    grpB_idx = np.reshape(np.where(lbl == grpb),(20,))
    A = disSMat[grpA_idx,:]
    B = A[:,grpB_idx]

    result = np.mean(B)

    if (grpa - grpb < 0.1):
        result = result * 20 / 19

    return result

def compareGroupsT(grpa,grpb,lbl,disSMat):
    #takes in two group indices, labels and dissimiliarity matrix, returns how similar they are as two groups
    grpA_idx = torch.where(lbl == grpa)[0]
    grpB_idx = torch.where(lbl == grpb)[0]
    A = disSMat[grpA_idx,:]
    B = A[:,grpB_idx]

    result = torch.mean(B)

    if (grpa - grpb < 0.1):
        result = result * 20 / 19

    return result

In [6]:
def obj_2_adj(fileName):
    #takes in file name, reads the .obj file and returns adjacency matrix scaled with edge values
    v, f = igl.read_triangle_mesh(fileName)
    distances = igl.all_pairs_distances(v,v,False)
    adJ = igl.adjacency_matrix(f).toarray() * distances
    return adJ


def obj_2_adj_noscale(fileName):
    #takes in file name, reads the .obj file and returns adjacency matrix
    v, f = igl.read_triangle_mesh(fileName)
    distances = igl.all_pairs_distances(v,v,False)
    adJ = igl.adjacency_matrix(f).toarray() * distances
    adJ = np.where(adJ != 0,1,0)
    return adJ

def adj_2_features(adj):
    #some arbitrary ad-hoc function for extracting features, for testing only

    #print([np.mean(adj),np.max(adj),np.std(adj)])
    #hist = np.histogram(adj,bins = np.arange(0,0.5,0.02),density=True)
    #generic_feat = np.array([np.mean(adj),np.max(adj),np.std(adj)])
    hist = np.histogram(adj,bins = 20,density=True)[0]
    #print(hist[0])
    return hist


In [7]:
def form_label_matrix(label_mat, group):
    rows = []
    for i in range(30):
        for j in range(15):
            # delete test obj from label matrix
            rows.append(group[i][j])

    label_mat = label_mat[:,rows]
    label_mat = label_mat[rows, :]
    print(label_mat.shape)
    return label_mat

In [8]:
def extract_node_feature(ver_coords,adj):#calculate all pairs distance (scaled adj matrix)
    pairDistance = torch.cdist(ver_coords,ver_coords,p = 2)
    adj_scaled = pairDistance * adj

    node_degs = torch.sum(adj,dim = 1)
    node_neigh_max = torch.max(adj_scaled,dim = 1).values
    node_neigh_sum = torch.sum(adj_scaled,dim = 1)
    #node_neigh_mean = node_neigh_sum / node_degs

    features = torch.stack([node_degs,node_neigh_max,node_neigh_sum],dim = 2)
    return features

In [9]:
def translate_by_group(group_mat,transformations):
    trans = transformations[:,0:1,:] * group_mat[:,:,0:1]
    
    for k in range(1,K):
        trans += transformations[:,k:k+1,:] * group_mat[:,:,k:k+1]
        
    return trans

In [10]:
def get_nodes(fileName):
    # returns a vertex narray
    v, f = igl.read_triangle_mesh(fileName)
    
    # resize the v so that all v's have [252 x 252]
    if v.shape[0] < 252:
        new_v = np.zeros([252, v.shape[1]])
        new_v[:v.shape[0],:] = v
        v = new_v
    
    return v, f


def K_mean_cluster(v, K):
    centroids, labels = scipy.cluster.vq.kmeans2(v, K, minit = "points")
    
    label_matrix = np.zeros((v.shape[0], K))
    
    for i in range(v.shape[0]):
        label_matrix[i,labels[i]] = 1
    
    return label_matrix, labels

In [11]:
def get_adj_from_f(faces):
    adj = np.zeros([252,252])
    adj_temp = igl.adjacency_matrix(f).toarray()
    adj[:adj_temp.shape[0],:adj_temp.shape[1]] = adj_temp
    return adj

In [12]:
def expand(origDir, newDir, rs):
    #first delete everything in newDir (so less likely to run into random errors)
    files = glob.glob(newDir + '*')
    for f in files:
        os.remove(f)
    
    #generate additional data and update label
    origLabel = np.array(readLbl(600,origDir + 'labels.txt'),dtype = int) #first take in the original labels
    newLabelFile = open(newDir + "labels.txt", "w")
    for i in range(600):
        #First create the new obj file
        origN = origDir + 'T' + str(i) + '.obj'

        for j in range(len(radiuss)):
            radius = rs[j]
            newN = newDir + 'T' + str(i+600*(j)) + '.obj'

            #newLabelFile.write("T" + str(i) + ".obj" + " " + str(origLabel[i]) + "\n")
            if not (write_new_files(origN,newN,radius)):
                print('Data generation failed on ' , i)

            newLabelFile.write("T" + str(i + 600*(j)) + ".obj" + " " + str(origLabel[i]) + "\n")


    newLabelFile.close()

In [13]:
def scramble(scramble_dir,num_swaps, num_objs):
    labels = np.array(readLbl(num_objs,scramble_dir+'labels.txt'))

    for swap_idx in range(num_swaps):
        idxA = random.randint(0,num_objs-1)
        idxB = random.randint(0,num_objs-1)

        if idxA != idxB:
            labelA = labels[idxA]
            labelB = labels[idxB]
            labels[idxA] = labelB
            labels[idxB] = labelA

            #change filenames
            os.rename(scramble_dir+'T' + str(idxA) + '.obj',scramble_dir+'temp.obj')
            os.rename(scramble_dir+'T' + str(idxB) + '.obj',scramble_dir+'T' + str(idxA) + '.obj')
            os.rename(scramble_dir+'temp.obj',scramble_dir+'T' + str(idxB) + '.obj')

    newLabelFile = open(scramble_dir + "labels.txt", "w")
    for i in range(num_objs):
            newLabelFile.write("T" + str(i) + ".obj" + " " + str(labels[i]) + "\n")


    newLabelFile.close()

In [14]:
def write_new_files(origin_name,new_name,radius):
    v,f = alternate_obj(origin_name,radius)
    file = open(new_name, "w")
    return igl.write_obj(new_name, v, f)

In [15]:
def alternate_obj(fName, radius):
    v, f = igl.read_triangle_mesh(fName)
    v = v + np.random.random(v.shape) *radius
    return v, f

In [16]:
def detect_nan(T):
    return torch.nonzero(torch.isnan(T.view(-1)))

In [27]:
def display_hist(output, label_mat, size):
    
    disMat = torch.cdist(output,output).flatten()
    print(disMat.shape)
    mask_same = torch.tensor(label_mat - torch.diag(torch.ones(size)),dtype=bool).flatten()
    print(mask_same.shape)
    
    
    mask_diff = torch.tensor(1 - label_mat,dtype=bool).flatten()
    sameComp = disMat[mask_same].detach().numpy()
    diffComp = disMat[mask_diff].detach().numpy()
    weightsSame = np.ones_like(sameComp) / len(sameComp)
    weightsDiff = np.ones_like(diffComp) / len(diffComp)
    
    print(np.mean(sameComp),np.mean(diffComp))
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    ax1.hist(sameComp,bins=np.arange(0,np.max(sameComp),0.01),weights=weightsSame,log = False)
    ax1.set_title('Same Comp')
    ax2.hist(diffComp,bins=np.arange(0,np.max(diffComp),0.01),weights=weightsDiff,log = False)
    ax2.set_title('Diff Comp')
    